In [2]:
import torch
import torch.nn as nn
import numpy as np
import math
import torchvision

1.torch.nn.**MaxPool2d**(kernel_size=2, stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False),<br />
2.torch.nn.**AvgPool2d**(kernel_size=2, stride=1, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None)

In [ ]:
x = torch.randn(3,32,32)

In [ ]:
l1 = torch.nn.MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, return_indices=False, ceil_mode=False)
#窗口大小2x2，每次滑动1格，周围不填充0，return_indices：返回值不包含最大值的索引
torch_out = l1(x)
my_out = my_Pool2d(2,1,x,'MaxPool2d')

In [ ]:
torch_out.shape

torch.Size([3, 16, 16])

In [ ]:
diff = torch.sum(torch_out-my_out)
diff

tensor(-9.3453e-05, dtype=torch.float64)

In [ ]:
l2=torch.nn.AvgPool2d(kernel_size=2, stride=1, padding=0, ceil_mode=False, count_include_pad=True, divisor_override=None)
#divisor_override:除法因子
torch_out = l2(x)
my_out = my_Pool2d(2,1,x,'AvgPool2d')

In [ ]:
diff = torch.sum(torch_out-my_out)
diff

tensor(-0.0001, dtype=torch.float64)

In [ ]:
def my_Pool2d(kernel_size,stride,inputx,method):
    output_width=int((inputx.shape[1]-kernel_size)/stride)+1
    output_height=int((inputx.shape[2]-kernel_size)/stride)+1
    output = np.zeros((len(inputx),output_width,output_height))
    for i in range(inputx.shape[0]):#channel
        for x in range(0,output_height,stride):
            for y in range(0,output_width,stride):
                if method=='MaxPool2d':
                    output[i,x,y]=np.max(inputx[i,x:x+kernel_size,y:y+kernel_size].numpy())
                if method=='AvgPool2d':
                    output[i,x,y]=np.mean(inputx[i,x:x+kernel_size,y:y+kernel_size].numpy())
    return np.round(output,5)

3.torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode=‘zeros')

In [ ]:
x = torch.randn(1,3,32,32)#add batch size

In [ ]:
l3=torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
#At groups=1, all inputs are convolved to all outputs
torch.nn.init.uniform_(l3.weight)
torch.nn.init.uniform_(l3.bias)
torch_out = l3(x)
my_out = my_Cov2d(3,1,6,l3.weight,l3.bias,x)

In [ ]:
diff = torch.sum(torch_out-torch.from_numpy(my_out))
diff

tensor(0.0003, dtype=torch.float64, grad_fn=<SumBackward0>)

In [ ]:
l3.bias.shape

torch.Size([6])

In [ ]:
l3.weight.shape

torch.Size([6, 3, 3, 3])

In [ ]:
torch_out.shape

torch.Size([1, 6, 30, 30])

In [ ]:
def my_Cov2d(kernel_size,stride,output_channels,weight,bias,inputx):
    output_width=int((inputx.shape[2]-kernel_size)/stride)+1
    output_height=int((inputx.shape[3]-kernel_size)/stride)+1
    output = np.zeros((1,output_channels,output_width,output_height))
    for i in range(output_channels):#channel
        for x in range(0,output_height,stride):
            for y in range(0,output_width,stride):
                for k in range(inputx.shape[1]):
                    output[0,i,x,y] += np.sum((inputx[0,k,x:x+kernel_size,y:y+kernel_size]*weight[i,k]).detach().numpy())
                output[0,i,x,y]+=bias[i]
       # output[0,i]+=bias
    return np.round(output,5)

4.torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode=‘zeros')

In [ ]:
l4=torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
torch.nn.init.uniform_(l4.weight)
torch.nn.init.uniform_(l4.bias)
torch_out = l4(x)

In [ ]:
l4.bias.shape

torch.Size([6])

In [ ]:
l4.weight.shape

torch.Size([6, 3, 5, 5])

In [ ]:
torch_out.shape

torch.Size([1, 6, 12, 12])

In [ ]:
l4.weight.shape[2]

5

In [ ]:
def pad_weight(ori_weight):
    pad_weight = np.zeros((ori_weight.shape[0],ori_weight.shape[1],2*ori_weight.shape[2]-1,2*ori_weight.shape[3]-1))
    for k in range(ori_weight.shape[0]):
        for c in range(ori_weight.shape[1]):
            for i in range(ori_weight.shape[2]):
                for j in range(ori_weight.shape[3]):
                    pad_weight[k,c,2*i,2*j]=ori_weight[k,c,i,j]
    return pad_weight

In [ ]:
weight = pad_weight(l4.weight)

In [ ]:
weight.shape

(6, 3, 9, 9)

In [ ]:
def my_Cov2d_dilation(kernel_size,stride,dilation,output_channels,weight,bias,inputx):
    output_width=int((inputx.shape[2]-dilation*(kernel_size-1)-1)/stride)+1
    output_height=int((inputx.shape[3]-dilation*(kernel_size-1)-1)/stride)+1
    output = np.zeros((1,output_channels,output_width,output_height))
    for i in range(output_channels):#channel
        for x in range(0,output_height*dilation,stride):
            for y in range(0,output_width*dilation,stride):
                for k in range(inputx.shape[1]):
                    output[0,i,x//2,y//2] += np.sum(inputx[0,k,x:x+2*kernel_size-1,y:y+2*kernel_size-1].numpy()*weight[i,k])
                output[0,i,x//2,y//2]+=bias[i]
       # output[0,i]+=bias
    return np.round(output,5)

In [ ]:
my_out = my_Cov2d_dilation(5,2,2,6,weight,l4.bias,x)
diff = torch.sum(torch_out-torch.from_numpy(my_out))
diff

tensor(3.2881e-05, dtype=torch.float64, grad_fn=<SumBackward0>)

5.torch.nn.ConvTranspose2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')

In [ ]:
l5=torch.nn.ConvTranspose2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, padding_mode='zeros')
torch.nn.init.uniform_(l5.weight)
torch.nn.init.uniform_(l5.bias)
torch_out = l5(x)

In [ ]:
l5.weight.shape

torch.Size([3, 4, 3, 3])

In [ ]:
l5.bias.shape

torch.Size([4])

In [ ]:
torch_out.shape

torch.Size([1, 4, 34, 34])

In [ ]:
def pad_inputx(dilation,kernel_size,x):
    pad = dilation * (kernel_size - 1)
    newx=np.zeros((1,x.shape[1],x.shape[2]+2*pad,x.shape[3]+2*pad))
    for i in range(x.shape[1]):
        newx[0,i]=np.pad(x[0,i], (pad, pad), 'constant')
    return newx

In [ ]:
x=pad_inputx(1,3,x)

In [ ]:
#output_width=int((inputx.shape[2]-kernel_size)/stride)+1 反过来
def my_ConvTranspose2d(kernel_size,stride,output_channels,weight,bias,inputx):
    output_width=int((inputx.shape[2]-kernel_size)/stride)+1
    output_height=int((inputx.shape[3]-kernel_size)/stride)+1
    output = np.zeros((1,output_channels,output_width,output_height))
    for i in range(output_channels):#4 output channel
        for x in range(0,output_height,stride):
            for y in range(0,output_width,stride):
                for k in range(inputx.shape[1]):#3 input channel
                    output[0,i,x,y] += np.sum(((inputx[0,k,x:x+kernel_size,y:y+kernel_size])*weight[k,i].detach().numpy()))
                output[0,i,x,y]+=bias[i]
    return np.round(output,5)

In [ ]:
my_out = my_ConvTranspose2d(3,1,4,l5.weight,l5.bias,x)#3,4,3,3
diff = torch.sum(torch_out-torch.from_numpy(my_out))
diff

tensor(-0.0002, dtype=torch.float64, grad_fn=<SumBackward0>)

6.torch.flatten(input, start_dim=0, end_dim=-1)

In [ ]:
x = torch.randn(3,32,32)

In [ ]:
torch_out=torch.flatten(x, start_dim=0, end_dim=-1)
torch_out.shape

torch.Size([3072])

In [ ]:
def my_flatten(inputx):
    output=[]
    for i in range(inputx.shape[0]):
        for x in range(inputx.shape[1]):
            for y in range(inputx.shape[2]):
                output.append(inputx[i,x,y])
    return np.round(output,5)

In [ ]:
my_out=my_flatten(x)
diff = np.sum(torch_out.numpy()-my_out)
diff

0.00011458222

7.torch.sigmoid(input, *, out=None)

In [ ]:
torch_out=torch.sigmoid(x,out=None)
torch_out.shape

torch.Size([3, 32, 32])

In [ ]:
def sigmoid(x):
        return 1 / (1 + math. exp(-x))
def my_sigmoid(inputx):
    for i in range(inputx.shape[0]):
        for x in range(inputx.shape[1]):
            for y in range(inputx.shape[2]):
                inputx[i,x,y]=1/(1 + math.exp(-inputx[i,x,y]))
    return np.round(inputx,5)

In [ ]:
my_out = my_sigmoid(x)
my_sigmoid(x).shape

torch.Size([3, 32, 32])

In [ ]:
diff = torch.sum(torch_out-my_out)
diff

tensor(4.3333e-05)

8.torchvision.ops.roi_pool(input: torch.Tensor, boxes: torch.Tensor, output_size: None, spatial_scale: float = 1.0)

In [5]:
#box [batch_id, x1, y1, x2, y2]，其中(x1, y1)为左上角，(x2, y2)为右下角。
x=torch.randn(1,3,32,32)
boxes=torch.tensor([[0,0,0,16,16]]).float()
l8=torchvision.ops.roi_pool(x, boxes, output_size=(5,5))
my_out=my_roi_pool(roi_start_w=0,roi_start_h=0,roi_end_w=16,roi_end_h=16,output_size=5,inputx=x)

In [6]:
l8.shape

torch.Size([1, 3, 5, 5])

In [4]:
def my_roi_pool(roi_start_w,roi_start_h,roi_end_w,roi_end_h,output_size,inputx):
    output = np.zeros((1,inputx.shape[1],output_size,output_size))
    
    roi_width = (roi_end_w - roi_start_w + 1)
    roi_height = (roi_end_h - roi_start_h + 1)

    bin_size_w = roi_width/output_size
    bin_size_h = roi_height/output_size
    
    for i in range(inputx.shape[1]):
        for y in range(output_size):
            for x in range(output_size):
                output[0,i,y,x]=np.max((inputx[0,i,int(roi_start_h+bin_size_h*(y)):math.ceil(roi_start_h+bin_size_h*(y+1)),int(roi_start_w+bin_size_w*(x)):math.ceil(roi_start_w+bin_size_w*(x+1))]).numpy())
    return np.round(output,5)

In [7]:
diff = torch.sum(l8-torch.from_numpy(my_out))
diff

tensor(7.9337e-06, dtype=torch.float64)

9.torch.nn.functional.batch_norm(input, running_mean, running_var, weight=None, bias=None, training=False, momentum=0.1, eps=1e-05)

In [ ]:
x=torch.randn(1,3,32,32)

In [ ]:
running_mean = torch.zeros(x.shape[1])
running_var = torch.ones(x.shape[1])
torch_out=torch.nn.functional.batch_norm(x,running_mean,running_var,weight=None,bias=None,training=False,momentum=0.1,eps=1e-05)

In [ ]:
torch_out.shape

torch.Size([1, 3, 32, 32])

In [ ]:
def my_batch_norm(inputx,running_mean,running_var):
    eps=1e-05
    for i in range(inputx.shape[1]):
        for x in range(inputx.shape[2]):
            for y in range(inputx.shape[3]):
                inputx[0,i,x,y]=(inputx[0,i,x,y]-running_mean[i])/torch.sqrt(running_var[i]+eps)
    return np.round(inputx,5)

In [ ]:
my_out = my_batch_norm(x,running_mean,running_var)
diff = torch.sum(torch_out-my_out)
diff

tensor(-0.0005)

10.torch.nn.functional.cross_entropy(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction=‘mean')

In [9]:
x=torch.randn(1,3,32,32)

In [10]:
target = torch.randint(0,2,(1,32,32))

In [11]:
l10=torch.nn.functional.cross_entropy(x, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
my_out=my_cross_entropy(x,target)
l10

tensor(1.3762)

In [12]:
my_out

tensor(1.3762)

In [8]:
def softmax(inputx):
    exp_xi = torch.exp(inputx[0])
    return exp_xi/torch.sum(exp_xi,dim=0)
def my_cross_entropy(inputx,target):
    softmax_inputx=softmax(inputx)
    loss=0
    for x in range(target.shape[1]):
        for y in range(target.shape[2]):
            class_i=target[0,x,y]
            loss_i=torch.log(softmax_inputx[class_i,x,y])
            loss-=loss_i
    loss = (loss)/(target.shape[1]*target.shape[2])
    return np.round(loss,5)

11.torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='mean')

In [ ]:
target=torch.randn(3,32,32)

In [ ]:
l11=torch.nn.functional.mse_loss(x, target, size_average=None, reduce=None, reduction='mean')
l11

tensor(1.9804)

In [ ]:
my_out = torch.mean((x-target)**2)
my_out

tensor(1.9804)